In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
igfold = 3
os.system('mkdir -p ./data_folds/fold_' + str(igfold))

0

In [2]:
!nvidia-smi

Fri Jan 13 13:33:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:85:00.0 Off |                    0 |
| N/A   39C    P0    71W / 163W |      0MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   

In [3]:
import cudf
import glob
import gc

from tqdm import tqdm

In [4]:
lb = False
path = '../data/'
type_weight = {0:1, 1:6, 2:3}
no_files = 3
path = '../../data/'
    
df_type = cudf.DataFrame({
    'type': ['clicks', 'carts', 'orders'],
    'type_': [0, 1, 2]
})

def list_in_chunks(files, no_chunks=10):
    out = [[] for _ in range(no_chunks)]
    for i, file in enumerate(files):
        out[i%no_chunks].append(file)
    return(out)

In [5]:
# !rm -rf ./data/
!mkdir -p ./data/

In [6]:
files = sorted(
    glob.glob('../../data/' + '/train/interim/*.parquet')
)
files_split = [glob.glob('../../data/test.parquet')] + [glob.glob('./data/xgb_train_x.parquet')] + list_in_chunks(files, no_chunks=len(files)//no_files)

In [7]:
len(files_split)

45

In [8]:
import pickle

sessions = pickle.load(open('./data/sessions_eval.pickle', 'rb'))
if igfold == 0:
    sess_eval = sessions[0]+sessions[1]
elif igfold == 1:
    sess_eval = sessions[2]+sessions[3]
elif igfold == 2:
    sess_eval = sessions[4]+sessions[5]
elif igfold == 3:
    sess_eval = sessions[6]+sessions[7]
elif igfold == 4:
    sess_eval = sessions[8]+sessions[9]
print(len(sess_eval))

356747


### 3) "Clicks" Co-visitation Matrix - Time Weighted

In [9]:
!rm -r /tmp/data/tmp2
!mkdir -p /tmp/data/tmp2/split

In [10]:
import cupy

In [11]:
out = []
for e, file in tqdm(enumerate(files_split)):
    df = cudf.read_parquet(file)
    df = df.merge(
        df_type,
        how='left',
        on='type'
    )
    if any([True for x in file if 'xgb_train_x.parquet' in x]):
        print(file)
        df = df[df['session'].isin(sess_eval)]
    else:
        df = df.loc[~(df['session'].isin(sess_eval))]
    df = df.sort_values(['session', 'ts'], ascending=[True, False])
    df['ts_2'] = df.groupby(['session']).ts.shift(1)
    df['diff_'] = df['ts_2']-df['ts']
    df['diff_'] = ((df['diff_']/1000)>(1*60*60)).fillna(0).astype('int8')
    df['new_session'] = df.groupby(['session']).diff_.cumsum()
    df.drop(['ts_2', 'diff_'], axis=1, inplace=True)
    df['session'] = df['session']*100+df['new_session']
    df.drop(['new_session'], axis=1, inplace=True)

    df['session'] = df['session'].astype('int32')
    df['aid'] = df['aid'].astype('int32')
    df.ts = (df.ts/1000).astype('int32')
    df.drop(['type'], axis=1, inplace=True)
    df = df.rename(columns={'type_': 'type'})
    df = df[df['type']==0]
    df = df.merge(
        df[['session']].groupby(['session']).size().reset_index().rename(columns={0: 'session_len'}),
        how='left',
        on=['session']
    )
    df['session_len'] = df['session_len'].astype('int16')
    df = df.sort_values(['session','ts'],ascending=[True,False])
    df = df.reset_index(drop=True)
    df['n'] = df.groupby('session').cumcount()
    df['n'] = df['n'].astype('int16')
    df.drop(['type'], axis=1, inplace=True)
    df = df.reset_index(drop=True)
    df = df.merge(
        df[['session', 'aid', 'ts', 'n']], 
        how='left',
        on='session'
    )
    
    df = df[df['n_x']!=df['n_y']]
    df = df[df['ts_x']!=df['ts_y']]
    df = df[
        ((df['ts_x']-df['ts_y']).abs()>30)|(df['aid_x']!=df['aid_y'])
    ]
    df = df[(df['aid_x']!=df['aid_y'])]
    df['diff_time'] = (df['ts_x']-df['ts_y']).abs()
    df.drop(['ts_x', 'ts_y'], axis=1, inplace=True)
    df['w_time'] = 1/cupy.log(df['diff_time']+2)
    df.drop(['diff_time'], inplace=True, axis=1)

    df['diff_pos'] = (df['n_x']-df['n_y']).abs()
    df['w_pos'] = 1/cupy.log(df['diff_pos']+2)
    df.drop(['diff_pos'], inplace=True, axis=1)

    df['w_sess'] = 1/cupy.log(df['session_len']+2)
    df.drop(['session_len'], inplace=True, axis=1)

    df['wgt'] = df['w_time']*df['w_pos']*df['w_sess']*(df['n_x']!=df['n_y']).astype('float32')
    
    df = df[['aid_x','aid_y','wgt']]
    
    gc.collect()
    df.wgt = df.wgt.astype('float32')
    df = df.groupby(['aid_x','aid_y']).wgt.agg(['sum', 'count']).reset_index()
    df = df[df['count']>1]
    df.to_parquet('/tmp/data/tmp2/split/split_' + str(e) + '.parquet')
    del df
    gc.collect()

1it [00:03,  3.08s/it]

['./data/xgb_train_x.parquet']


45it [01:43,  2.31s/it]


In [12]:
files_split = glob.glob('/tmp/data/tmp2/split/split_*.parquet')
df = cudf.read_parquet(files_split[0])
for file in files_split[1:]:
    print(file)
    df2 = cudf.read_parquet(file)
    df = cudf.concat([
        df,
        df2
    ])
    gc.collect()
    df = df.groupby(['aid_x','aid_y']).sum()
    df = df.reset_index()
    del df2
    gc.collect()

/tmp/data/tmp2/split/split_9.parquet
/tmp/data/tmp2/split/split_36.parquet
/tmp/data/tmp2/split/split_39.parquet
/tmp/data/tmp2/split/split_25.parquet
/tmp/data/tmp2/split/split_24.parquet
/tmp/data/tmp2/split/split_6.parquet
/tmp/data/tmp2/split/split_28.parquet
/tmp/data/tmp2/split/split_35.parquet
/tmp/data/tmp2/split/split_11.parquet
/tmp/data/tmp2/split/split_14.parquet
/tmp/data/tmp2/split/split_32.parquet
/tmp/data/tmp2/split/split_44.parquet
/tmp/data/tmp2/split/split_31.parquet
/tmp/data/tmp2/split/split_42.parquet
/tmp/data/tmp2/split/split_12.parquet
/tmp/data/tmp2/split/split_19.parquet
/tmp/data/tmp2/split/split_34.parquet
/tmp/data/tmp2/split/split_23.parquet
/tmp/data/tmp2/split/split_16.parquet
/tmp/data/tmp2/split/split_37.parquet
/tmp/data/tmp2/split/split_30.parquet
/tmp/data/tmp2/split/split_22.parquet
/tmp/data/tmp2/split/split_29.parquet
/tmp/data/tmp2/split/split_27.parquet
/tmp/data/tmp2/split/split_13.parquet
/tmp/data/tmp2/split/split_20.parquet
/tmp/data/tmp2

In [13]:
files = sorted(glob.glob('../../data/' + '/train/interim/*.parquet')) + glob.glob('../../data/test.parquet') + glob.glob('./data/xgb_train_x.parquet')
out = []
for file in files:
    df_tmp = cudf.read_parquet(file)
    if 'xgb_train_x.parquet' in file:
        print(file)
        df_tmp = df_tmp[df_tmp['session'].isin(sess_eval)]
    else:
        df_tmp = df_tmp[~(df_tmp['session'].isin(sess_eval))]
    out.append(df_tmp)

./data/xgb_train_x.parquet


In [14]:
df_aid = cudf.concat(out)
del out
gc.collect()

0

In [15]:
df_aid = df_aid['aid'].value_counts().reset_index()
df_aid = df_aid.reset_index()
df_aid.columns = ['aid_', 'aid', 'count']
df_aid.drop(['aid_'], inplace=True, axis=1)

In [16]:
df_aid.columns = ['aid_x', 'count_x']
df = df.merge(
    df_aid,
    how='left',
    on='aid_x'
)
df_aid.columns = ['aid_y', 'count_y']
df = df.merge(
    df_aid,
    how='left',
    on='aid_y'
)

In [17]:
del df_aid
gc.collect()

25

In [18]:
df = df.reset_index(drop=True)

In [19]:
df

,aid_x,aid_y,sum,count,count_x,count_y
0,334325,1019736,0.125056,2,3023,29423
1,480362,763063,0.049614,4,132,121
2,1718422,472478,0.066710,4,410,551
3,784887,1695413,0.079596,2,6798,28121
4,1008756,1228666,0.111463,16,58,735
...,...,...,...,...,...,...
335569839,1451845,1502103,0.059438,2,63,324
335569840,1259743,776056,0.039153,2,721,13
335569841,1124603,856880,0.023536,4,1438,17
335569842,768777,946621,0.017826,2,130,8632


In [20]:
gc.collect()

165

In [21]:
import cupy

In [22]:
df['wgt'] = df['sum']/(cupy.sqrt((df['count_x'].fillna(0)+1).values)*cupy.sqrt((df['count_y'].fillna(0)+1).values))

In [23]:
df.drop(['sum', 'count_x', 'count_y'], axis=1, inplace=True)

In [24]:
df_pd = df.to_pandas()

In [25]:
df_pd.to_parquet('./data_folds/fold_' + str(igfold) + '/top_20_recsys2022_weighted_v5.parquet')

In [26]:
del df, df_pd
gc.collect()

0

### Generate Candidates

In [9]:
!nvidia-smi

Fri Jan 13 13:33:51 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.119.04   Driver Version: 450.119.04   CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:85:00.0 Off |                    0 |
| N/A   39C    P0    71W / 163W |    646MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-SXM2...  On   | 00000000:8A:00.0 Off |                    0 |
| N/A   

In [10]:
import cudf
import numpy as np
import gc
import os

import glob

In [11]:
def get_candidates(df, co, prefix, treshhold=2, treshhold_topn=15):
    df = df.merge(
        co[co['n']<treshhold_topn][['aid', 'cand', 'wgt', 'count']],
        how='left',
        on='aid'
    )
    df['wgt'] = df['wgt'].fillna(0)
    df['count'] = df['count'].fillna(0)
    df.drop(['aid', 'ts', 'type'], axis=1, inplace=True)
    df['wgt1'] = df['wgt'].values
    df['count1'] = df['count'].values
    df = df.groupby(['session', 'cand']).agg({
        'dummy': 'sum',
        'wgt': 'sum',
        'wgt1': 'max',
        'count': 'sum',
        'count1': 'max'
    }).reset_index()
    df.columns = [
        'session', 'cand', 
        prefix+'_num_' + str(treshhold_topn),
        prefix+'_wgt_' + str(treshhold_topn),
        prefix+'_wgt_max_' + str(treshhold_topn),
        prefix+'_count_' + str(treshhold_topn),
        prefix+'_count_max_' + str(treshhold_topn),
    ]
    return(df)

def list_in_chunks(files, no_chunks=10):
    out = [[] for _ in range(no_chunks)]
    for i, file in enumerate(files):
        out[i%no_chunks].append(file)
    return(out)

In [12]:
prefix = 'weighted'

In [13]:
for dataset in ['train', 'sub']:
    for treshhold_topn in [30]:
        print(dataset, treshhold_topn)
        os.system('mkdir -p ./data_folds/fold_' + str(igfold) + '/candidates/' + dataset + '/' + prefix + '_' + str(treshhold_topn))
        co = cudf.read_parquet('./data_folds/fold_' + str(igfold) + '/top_20_recsys2022_weighted_v5.parquet')[['aid_x', 'aid_y', 'wgt', 'count']]
        co = co.sort_values(['aid_x','wgt'],ascending=[True,False])
        co = co.reset_index(drop=True)
        co['n'] = co.groupby('aid_x').aid_y.cumcount()
        co.columns = ['aid', 'cand', 'wgt', 'count', 'n']
        if dataset == 'train':
            df = cudf.read_parquet('./data/xgb_train_x.parquet')
            df = df[df['session'].isin(sess_eval)]
        else:
            df = cudf.read_parquet('../../data/test.parquet')
        df = df.sort_values(['session', 'ts'], ascending=[True, False])
        df['dummy'] = 1
        df['rank'] = df.groupby(['session']).dummy.cumsum()
        df['score'] = 1/df['rank']
        session_lists = list_in_chunks(df['session'].drop_duplicates().to_pandas().values.tolist(), no_chunks=10)
        out = []
        for session_list in session_lists:
            df_tmp = df[df['session'].isin(session_list)]
            df_tmp = get_candidates(df_tmp, co, prefix, treshhold=2, treshhold_topn=treshhold_topn)
            out.append(df_tmp)
        df = cudf.concat(out)
        df['session'] = df['session'].astype('int32')
        df['cand'] = df['cand'].astype('int32')
        df[prefix + '_num_'+ str(treshhold_topn)] = df[prefix + '_num_'+ str(treshhold_topn)].astype('int16')
        df[prefix + '_wgt_'+ str(treshhold_topn)] = df[prefix + '_wgt_'+ str(treshhold_topn)].astype('float32')
        df[prefix + '_wgt_max_'+ str(treshhold_topn)] = df[prefix + '_wgt_max_'+ str(treshhold_topn)].astype('float32')
        df[prefix + '_count_'+ str(treshhold_topn)] = df[prefix + '_count_'+ str(treshhold_topn)].astype('float32')
        df[prefix + '_count_max_'+ str(treshhold_topn)] = df[prefix + '_count_max_'+ str(treshhold_topn)].astype('float32')
        df.to_parquet('./data_folds/fold_' + str(igfold) + '/candidates/' + dataset + '/' + prefix + '_' + str(treshhold_topn) + '/cand.parquet')
        del df
        gc.collect()

train 30
sub 30
